In [ ]:
cd ./drive/My\ Drive

/content/drive/My Drive


In [ ]:
cd ./FCHarDNet_CamVid/

/content/drive/My Drive/FCHarDNet_CamVid


In [ ]:
ls

CamVid@     ptsemseg/           runs_cityscape_fe/          test.py
configs/    pytorch_bn_fusion/  runs_cityscape_freezing3/   train.py
data_road/  README.md           runs_cityscapes/            validate.py
FE.ipynb    runs/               runs_cityscapes_freezing2/  weights/
pic/        runs_camvid/        runs_kitti/


In [ ]:
!pip install scipy==1.1.0


     |████████████████████████████████| 31.2MB 101kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
import torch
torch.__version__

'1.6.0+cu101'

In [ ]:
!pip install torchstat

In [ ]:
config = "./configs/hardnet.yml"

In [ ]:
with open(config) as fp:
  cfg = yaml.load(fp)

In [ ]:
cfg

{'data': {'dataset': 'camvid',
  'img_cols': 1024,
  'img_rows': 1024,
  'path': '/content/drive/My Drive/FCHarDNet_CamVid/CamVid',
  'sbd_path': '/content/drive/My Drive/FCHarDNet_CamVid/CamVid',
  'train_split': 'train',
  'val_split': 'val'},
 'model': {'arch': 'hardnet'},
 'training': {'augmentations': {'hflip': 0.5, 'rscale_crop': [1024, 1024]},
  'batch_size': 16,
  'finetune': 'None',
  'loss': {'loss_th': 0.3,
   'min_K': 4096,
   'name': 'bootstrapped_cross_entropy',
   'size_average': True},
  'lr_schedule': {'max_iter': 10000, 'name': 'poly_lr'},
  'n_workers': 8,
  'optimizer': {'lr': 0.1,
   'momentum': 0.9,
   'name': 'sgd',
   'weight_decay': 0.0005},
  'print_interval': 10,
  'resume': None,
  'train_iters': 10000,
  'val_interval': 50}}

In [ ]:
pip install tensorboardX

     |████████████████████████████████| 317kB 12.6MB/s 


In [ ]:
import os
import yaml
import time
import shutil
import torch
import random
import argparse
import numpy as np
import torch.nn as nn

from torch.utils import data
from tqdm import tqdm

from ptsemseg.models import get_model
from ptsemseg.loss import get_loss_function
from ptsemseg.loader import get_loader
from ptsemseg.utils import get_logger
from ptsemseg.metrics import runningScore, averageMeter
from ptsemseg.augmentations import get_composed_augmentations
from ptsemseg.schedulers import get_scheduler
from ptsemseg.optimizers import get_optimizer

from tensorboardX import SummaryWriter

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight)

In [ ]:
def train(cfg, writer, logger):

    # Setup seeds
    torch.manual_seed(cfg.get("seed", 1337))
    torch.cuda.manual_seed(cfg.get("seed", 1337))
    np.random.seed(cfg.get("seed", 1337))
    random.seed(cfg.get("seed", 1337))

    # Setup device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Setup Augmentations
    augmentations = cfg["training"].get("augmentations", None)
    data_aug = get_composed_augmentations(augmentations)

    # Setup Dataloader
    data_loader = get_loader(cfg["data"]["dataset"])
    data_path = cfg["data"]["path"]

    t_loader = data_loader(
        data_path,
        is_transform=True,
        split=cfg["data"]["train_split"],
        img_size=(cfg["data"]["img_rows"], cfg["data"]["img_cols"]),
        augmentations=data_aug,
    )

    v_loader = data_loader(
        data_path,
        is_transform=True,
        split=cfg["data"]["val_split"],
        img_size=(1024,2048),
    )

    n_classes = t_loader.n_classes
    trainloader = data.DataLoader(
        t_loader,
        batch_size=cfg["training"]["batch_size"],
        num_workers=cfg["training"]["n_workers"],
        shuffle=True,
    )

    valloader = data.DataLoader(
        v_loader, batch_size=cfg["training"]["batch_size"], num_workers=cfg["training"]["n_workers"]
    )

    # Setup Metrics
    running_metrics_val = runningScore(n_classes)

    # Setup Model
    model = get_model(cfg["model"], 19).to(device)

    
    total_params = sum(p.numel() for p in model.parameters())
    print( 'Parameters:',total_params )

    #model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    #model.apply(weights_init)
    pretrained_path='./weights/hardnet_cityscapes_best_model.pkl'
    weights = convert_state_dict(torch.load(pretrained_path)['model_state'])
    model.load_state_dict(weights)
    model.finalConv = nn.Conv2d(48, 2, kernel_size=(1, 1), stride=(1, 1))
    for name, child in model.named_children():
      if (name != 'finalConv' and name != 'conv1x1_up'):
        for param in child.parameters():
          #print(child)
          param.requires_grad = False
    model.to(device)

    # Setup optimizer, lr_scheduler and loss function
    optimizer_cls = get_optimizer(cfg)
    optimizer_params = {k: v for k, v in cfg["training"]["optimizer"].items() if k != "name"}

    optimizer = optimizer_cls(model.parameters(), **optimizer_params)
    print("Using optimizer {}".format(optimizer))

    scheduler = get_scheduler(optimizer, cfg["training"]["lr_schedule"])

    loss_fn = get_loss_function(cfg)
    print("Using loss {}".format(loss_fn))

    start_iter = 0
    if cfg["training"]["resume"] is not None:
        if os.path.isfile(cfg["training"]["resume"]):
            logger.info(
                "Loading model and optimizer from checkpoint '{}'".format(cfg["training"]["resume"])
            )
            checkpoint = torch.load(cfg["training"]["resume"])
            model.load_state_dict(checkpoint["model_state"])
            #optimizer.load_state_dict(checkpoint["optimizer_state"])
            #scheduler.load_state_dict(checkpoint["scheduler_state"])
            start_iter = checkpoint["epoch"]
            logger.info(
                "Loaded checkpoint '{}' (iter {})".format(
                    cfg["training"]["resume"], checkpoint["epoch"]
                )
            )
        else:
            logger.info("No checkpoint found at '{}'".format(cfg["training"]["resume"]))

    if cfg["training"]["finetune"] is not None:
        if os.path.isfile(cfg["training"]["finetune"]):
            logger.info(
                "Loading model and optimizer from checkpoint '{}'".format(cfg["training"]["finetune"])
            )
            checkpoint = torch.load(cfg["training"]["finetune"])
            model.load_state_dict(checkpoint["model_state"])

    val_loss_meter = averageMeter()
    time_meter = averageMeter()

    best_iou = -100.0
    i = start_iter
    flag = True
    loss_all = 0
    loss_n = 0
    while i <= cfg["training"]["train_iters"] and flag:
        for (images, labels) in trainloader:
            i += 1
            start_ts = time.time()
            scheduler.step()
            model.train()
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)

            loss = loss_fn(input=outputs, target=labels)
            loss.backward()
            optimizer.step()
            c_lr = scheduler.get_lr()

            time_meter.update(time.time() - start_ts)
            loss_all += loss.item()
            loss_n += 1
            
            if (i + 1) % cfg["training"]["print_interval"] == 0:
                fmt_str = "Iter [{:d}/{:d}]  Loss: {:.4f}  Time/Image: {:.4f}  lr={:.6f}"
                print_str = fmt_str.format(
                    i + 1,
                    cfg["training"]["train_iters"],
                    loss_all / loss_n,
                    time_meter.avg / cfg["training"]["batch_size"],
                    c_lr[0],
                )
                

                print(print_str)
                logger.info(print_str)
                writer.add_scalar("loss/train_loss", loss.item(), i + 1)
                time_meter.reset()

            if (i + 1) % cfg["training"]["val_interval"] == 0 or (i + 1) == cfg["training"][
                "train_iters"
            ]:
                torch.cuda.empty_cache()
                model.eval()
                loss_all = 0
                loss_n = 0
                with torch.no_grad():
                    for i_val, (images_val, labels_val) in tqdm(enumerate(valloader)):
                        images_val = images_val.to(device)
                        labels_val = labels_val.to(device)

                        outputs = model(images_val)
                        val_loss = loss_fn(input=outputs, target=labels_val)

                        pred = outputs.data.max(1)[1].cpu().numpy()
                        gt = labels_val.data.cpu().numpy()

                        running_metrics_val.update(gt, pred)
                        val_loss_meter.update(val_loss.item())

                writer.add_scalar("loss/val_loss", val_loss_meter.avg, i + 1)
                logger.info("Iter %d Val Loss: %.4f" % (i + 1, val_loss_meter.avg))

                score, class_iou = running_metrics_val.get_scores()
                for k, v in score.items():
                    print(k, v)
                    logger.info("{}: {}".format(k, v))
                    writer.add_scalar("val_metrics/{}".format(k), v, i + 1)

                for k, v in class_iou.items():
                    logger.info("{}: {}".format(k, v))
                    writer.add_scalar("val_metrics/cls_{}".format(k), v, i + 1)

                val_loss_meter.reset()
                running_metrics_val.reset()
                
                state = {
                      "epoch": i + 1,
                      "model_state": model.state_dict(),
                      "optimizer_state": optimizer.state_dict(),
                      "scheduler_state": scheduler.state_dict(),
                }
                save_path = os.path.join(
                    writer.file_writer.get_logdir(),
                    "{}_{}_checkpoint.pkl".format(cfg["model"]["arch"], cfg["data"]["dataset"]),
                )
                torch.save(state, save_path)

                if score["Mean IoU : \t"] >= best_iou:
                    best_iou = score["Mean IoU : \t"]
                    state = {
                        "epoch": i + 1,
                        "model_state": model.state_dict(),
                        "best_iou": best_iou,
                    }
                    save_path = os.path.join(
                        writer.file_writer.get_logdir(),
                        "{}_{}_best_model.pkl".format(cfg["model"]["arch"], cfg["data"]["dataset"]),
                    )
                    torch.save(state, save_path)
                torch.cuda.empty_cache()

            if (i + 1) == cfg["training"]["train_iters"]:
                flag = False
                break

In [ ]:
run_id = random.randint(1, 100000)
logdir = os.path.join("runs_cityscapes_freezing2", os.path.basename("./configs/hardnet.yml")[:-4], "cur1")
writer = SummaryWriter(log_dir=logdir)

print("RUNDIR: {}".format(logdir))
shutil.copy("./configs/hardnet.yml", logdir)

logger = get_logger(logdir)
logger.info("Let the games begin")

train(cfg, writer, logger)


RUNDIR: runs_cityscapes_freezing2/hardnet/cur1


INFO:ptsemseg:Using bootstrapped_cross_entropy with {'min_K': 4096, 'loss_th': 0.3, 'size_average': True} params


Parameters: 4119257
Using optimizer SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005
)
Using loss functools.partial(<function bootstrapped_cross_entropy2d at 0x7fd98a3cad90>, min_K=4096, loss_th=0.3, size_average=True)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
INFO:ptsemseg:Iter [10/10000]  Loss: 1.6349  Time/Image: 0.1165  lr=0.099919


Iter [10/10000]  Loss: 1.6349  Time/Image: 0.1165  lr=0.099919


...

Overall Acc: 	 0.9761784355518885
Mean Acc : 	 0.9744886938679901
FreqW Acc : 	 0.9538337190010204
Mean IoU : 	 0.9443580746165523


INFO:ptsemseg:Iter [4010/10000]  Loss: 0.6471  Time/Image: 0.0916  lr=0.063059


Iter [4010/10000]  Loss: 0.6471  Time/Image: 0.0916  lr=0.063059


INFO:ptsemseg:Iter [4020/10000]  Loss: 0.6475  Time/Image: 0.0665  lr=0.062965


Iter [4020/10000]  Loss: 0.6475  Time/Image: 0.0665  lr=0.062965
